In [1]:
# 1. further clean the dataset based on df_all_1.pkl (categorical one)
# 2. impute missing values
# 3. numeric encoding for the categorical featutes
# 4. fit lightgbm model


In [2]:
import numpy as np
import pandas as pd

In [1]:
# df = pd.read_pickle('df_all_1.pkl')
# df.head()

In [2]:
# for col in df.columns:
#     print('column name: ', col)
#     print(df[col].isnull().sum()/len(df))
#     print('\n')

In [4]:
#1.impute missing values for numerical and categorical features

In [3]:
# df.select_dtypes(exclude = ['object']).tail()

In [4]:
# for col in df.select_dtypes(exclude = ['object']).columns:
#     print('column name: ', col)
#     print(df[col].isnull().sum())
#     print((df[col] == -1).sum())
#     print('\n')

In [48]:
#impute numerical feature missing values NAN with median

for col in df.select_dtypes(exclude = ['object']).columns:
    df[col].fillna(df[col].median(), inplace = True)

In [5]:
# for col in df.select_dtypes(exclude = ['object']).columns:
#     print('column name', col)
#     print(df[col].isnull().sum())
#     print((df[col] == -1).sum())
#     print('\n')

In [6]:
# pd.DataFrame(df['inc_descrip'].value_counts())

In [51]:
df['inc_descrip'].value_counts().index.tolist()[0]

'Collision with in-transit vehicle'

In [7]:
# impute categorical feature with the highest frequent level

# for col in df.select_dtypes(include = ['object']):
#     print('column name: ', col)
#     print(df[col].isnull().sum())
#     print('\n')

In [53]:
for col in df.select_dtypes(include = ['object']):
    df[col].fillna(df[col].value_counts().index.tolist()[0], inplace = True)

In [8]:
# for col in df.columns:
#     print('column name: ', col)
#     print(df[col].isnull().sum())
#     print('\n')

In [55]:
names = df['inc_descrip'].unique()
names

array(['Instability', 'Collision with in-transit vehicle',
       'Collision with lifeform ', 'Non-roadway',
       'Collision with parked vehicle', 'Dynamic',
       'Collision with safety barrier', 'Collision with object',
       'Hit by other vehicle', 'Non-collision', 'Pothole', 'unknown'],
      dtype=object)

In [56]:
mapping = {names[i]: i for i in range(0, len(names))}

In [57]:
df['inc_descrip'].map(mapping).head()

0    0
1    0
2    0
3    0
4    1
Name: inc_descrip, dtype: int64

In [58]:
df['inc_descrip'].head()

0                          Instability
1                          Instability
2                          Instability
3                          Instability
4    Collision with in-transit vehicle
Name: inc_descrip, dtype: object

In [59]:
df.select_dtypes(include = ['object']).columns

Index(['inc_day', 'inc_daylight', 'inc_weather', 'veh_roadcond',
       'veh_roadslope', 'veh_lanediv', 'occ_sex', 'occ_role', 'occ_position',
       'occ_seatbelt', 'occ_airbag', 'occ_alcohol', 'veh_owner', 'veh_type',
       'veh_style', 'veh_weight', 'veh_manu', 'veh_drive', 'veh_fuel',
       'veh_trailer', 'drv_licence', 'drv_status', 'drv_alcohol', 'inc_type',
       'inc_speeding', 'veh_braking', 'veh_stability', 'veh_movement',
       'veh_impact', 'inc_descrip', 'veh_descrip', 'veh_damage', 'mod_lb'],
      dtype='object')

In [9]:
# col_names = ['inc_day', 'inc_daylight', 'inc_weather', 'veh_roadcond',
#        'veh_roadslope', 'veh_lanediv', 'occ_sex', 'occ_role', 'occ_position',
#        'occ_seatbelt', 'occ_airbag', 'occ_alcohol', 'veh_owner', 'veh_type',
#        'veh_style', 'veh_weight', 'veh_manu', 'veh_drive', 'veh_fuel',
#        'veh_trailer', 'drv_licence', 'drv_status', 'drv_alcohol', 'inc_type',
#        'inc_speeding', 'veh_braking', 'veh_stability', 'veh_movement',
#        'veh_impact', 'inc_descrip', 'veh_descrip', 'veh_damage']
# col_names

In [61]:
# numeric encoding the categorical variables:

for col in col_names:
    levels = df[col].unique()
    mapping = {levels[i]: i for i in range(0, len(levels))}
    df[col] = df[col].map(mapping)

In [10]:
# df.head()

In [11]:
# for col in df.columns:
#     print('column names', col)
#     print(df[col].isnull().sum())
#     print('\n')

In [64]:
df.to_pickle('df_all_3.pkl')

In [65]:
# LightGBM

In [66]:
y = pd.read_pickle('resp.pkl')
len(y)

145063

In [68]:
X_df_train = df.loc[df['mod_lb'] == 'train', :]
X_df_test = df.loc[df['mod_lb'] == 'test', :]

In [69]:
print(X_df_train.shape)
print(X_df_test.shape)

(145063, 65)
(289671, 65)


In [70]:
X_df_train.drop(['mod_lb'], axis = 1, inplace = True)
X_df_test.drop(['mod_lb'], axis = 1, inplace = True)

C:\Users\LUY1\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [71]:
print(X_df_train.shape)
print(X_df_test.shape)

(145063, 64)
(289671, 64)


In [73]:
# random sampling based on the 'incident_id'

index = np.unique(X_df_train['incident_id'])
len(index)

64149

In [79]:
index

array([     2,      4,      5, ..., 128186, 128187, 128188], dtype=int64)

In [110]:
np.max(index)

128188

In [111]:
np.min(index)

2

In [80]:
from sklearn.model_selection import train_test_split

index_train, index_valid = train_test_split(index, test_size = 0.2, random_state = 42)
print(len(index_train))
print(len(index_valid))

51319
12830


In [81]:
51319 + 12830

64149

In [82]:
index_train

array([ 95160,  22227,  10074, ...,   1702,  31771, 112882], dtype=int64)

In [12]:
# X_df_train.head()

In [ ]:
X_df_train.set_index(['incident_id'], inplace = True)

In [13]:
# X_df_train.head()

In [14]:
# X_df_train.loc[[2, 4], :]

In [98]:
X_train = X_df_train.loc[index_train, :]
X_valid = X_df_train.loc[index_valid, :]

In [15]:
# X_train.head()

In [16]:
# X_train.reset_index().head()

In [103]:
X_train.reset_index(inplace = True)

In [17]:
# X_train.head()

In [18]:
# X_valid.reset_index(inplace = True)
# X_valid.head()

In [19]:
# read train dataset

# pp_data_train = pd.read_csv('pp_data_train.csv')
# pp_data_train.head()

In [22]:
# y_all = pp_data_train[['row_id', 'incident_id', 'vehicle_id', 'occupant_id', 'injury']]
# y_all.head()

In [149]:
mapping_y =  {'Unharmed':0, 'Possible':1, 'Minor':2, 'Major':3, 'Fatal':4}
print(mapping_y)

{'Unharmed': 0, 'Possible': 1, 'Minor': 2, 'Major': 3, 'Fatal': 4}


In [23]:
# y_all['injury'] = y_all['injury'].map(mapping_y)
# y_all.head()

In [151]:
y_all.isnull().sum()

row_id         0
incident_id    0
vehicle_id     0
occupant_id    0
injury         0
dtype: int64

In [153]:
y_all.shape

(145063, 5)

In [154]:
y_all.to_pickle('y_resp.pkl')

In [160]:
row_id_train = X_train['row_id']
row_id_valid = X_valid['row_id']

In [161]:
min(row_id_train)

2

In [162]:
max(row_id_train)

289665

In [164]:
min(row_id_valid)

44

In [165]:
max(row_id_valid)

289666

In [166]:
y_all.set_index(['row_id'], inplace = True)

In [158]:
y_all['row_i

2

In [159]:
y_all['row_id'].max()

289666

In [24]:
# y_all.head()

In [137]:
y_train = y_all.loc[index_train,]
y_valid = y_all.loc[index_valid,]
#y_all.loc[[2,3],:]

C:\Users\LUY1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  """Entry point for launching an IPython kernel.
C:\Users\LUY1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  


In [143]:
max(index_train)

128187

In [144]:
min(index_train)

2

In [146]:
max(index_valid)

128188

In [147]:
min(index_valid)

14

In [25]:
# y_train.head()

In [26]:
# y_train.reset_index()

In [27]:
# y_valid.reset_index()

In [ ]:
y_train.drop(['row_id', ''], inplace = True)
y_valid.drop([], inplace = True)

In [28]:
#X_df_train.loc[X_df_train['incident_id'] == index_train, :]

# X_df_train['incident_id'][index_train]

In [ ]:
# X_train, X_valid, X_test

X_train = X_df_train.loc[X_df_train['incident_id'] == index]